In [10]:
import numpy as np
import tensornetwork as tn
import time
import datetime
from qiskit import QuantumCircuit
from qiskit.quantum_info.operators import Operator
from func_timeout import func_set_timeout
import pandas as pd

In [11]:
def apply_gate(qubit_edges, gate, operating_qubits):
    op = tn.Node(gate)
    for i, bit in enumerate(operating_qubits):
        tn.connect(qubit_edges[bit], op[i])
        qubit_edges[bit] = op[i + len(operating_qubits)]

In [12]:
def matrix_2_np(U,q):
    if len(q)!=2:
        return U
    U_new=np.zeros((2, 2, 2, 2), dtype=complex)
    for k1 in range(2):
        for k2 in range(2):
            for k3 in range(2):
                for k4 in range(2):
                    U_new[k1][k2][k3][k4]=U[2*k2+k1][2*k4+k3]
    return U_new

def np_2_matrix(U,q_num):
    U_new=np.zeros((2**q_num,2**q_num), dtype=complex)
    for k1 in range(2**q_num):
        for k2 in range(2**q_num):
            b1=[int(b1) for b1 in list(bin(k1)[2:])]
            for k3 in range(q_num-len(b1)):
                b1.insert(0,0)
            b2=[int(b2) for b2 in list(bin(k2)[2:])]
            for k3 in range(q_num-len(b2)):
                b2.insert(0,0)            
            temp=U
            for b in b1:
                temp=temp[b]
            for b in b2:
                temp=temp[b]
            U_new[k1][k2]=temp
    return U_new

In [13]:
def get_real_qubits_num(cir):
    gates=cir.data
    q=0
    for k in range(len(gates)):
        q=max(q,max([qbit.index for qbit in gates[k][1]]))
    return q+1

In [14]:
@func_set_timeout(3600)
def Simulation_with_TensorNetwork(cir):
    all_nodes = []
    with tn.NodeCollection(all_nodes):
        qubits_num=get_real_qubits_num(cir)
#         print(qubits_num)
        state_nodes = [
            tn.Node(np.eye(2, dtype=complex)) for _ in range(qubits_num)
        ]
        
        qubits0=[node[0] for node in state_nodes]
        qubits = [node[1] for node in state_nodes]
        gates=cir.data
        for k in range(len(gates)):
            U=Operator(gates[k][0]).data
            q=[qbit.index for qbit in gates[k][1]]
            if len(q)==2:
                U=matrix_2_np(U,q)
#             print(k,U)
            apply_gate(qubits, U, q)
        edge_order=qubits0+qubits
#         print(edge_order)
        result = tn.contractors.auto(all_nodes, output_edge_order=edge_order).tensor
        return result

In [15]:
def Simulation_with_time_out(cir):
    try:
        return Simulation_with_TensorNetwork(cir)
    except:
        return 0

In [16]:
path='Benchmarks/'
file_name="4_49_16.qasm"
cir=QuantumCircuit.from_qasm_file(path+file_name)
t_start= time.time()
U=Simulation_with_TensorNetwork(cir)
t_end=time.time()
print(t_end-t_start)
# qubits_num=get_real_qubits_num(cir)
# U=np_2_matrix(U,qubits_num)
# print(U)

0.3119978904724121


In [17]:
# path='Benchmarks2/'
# file_list = open('test.txt','r')
# excel_path = 'data.xlsx'
# result=[]
# with open('computed.txt','r') as f:
#     for line in f:
#         result.append(line[:-1])

# data = ["New"]
# data=np.array(data).reshape(1,len(data))
# df = pd.read_excel(excel_path, header=None)
# ds = pd.DataFrame(data)
# df = df.append(ds, ignore_index=True)
# df.to_excel(excel_path, index=False, header=False)

# for file in file_list.readlines():
#     file_name=file[:-1]

#     if file_name in result:
#         continue
#     print(file_name)
    
#     cir=QuantumCircuit.from_qasm_file(path+file_name)
#     qubits_num=get_real_qubits_num(cir)
    
#     data = []
#     data.append(file_name)
#     data.append(qubits_num)
    
#     time_now = datetime.datetime.now()
# #     print(time_now.strftime('%m.%d-%H:%M:%S'))
    
#     t_start = time.time()
#     res=Simulation_with_time_out(cir)
#     run_time=time.time()-t_start
# #     print(run_time)
#     data.append(run_time)
#     if isinstance(res,int):
#         data.append(-1)
        
    
#     data=np.array(data).reshape(1,len(data))
#     df = pd.read_excel(excel_path, header=None)
#     ds = pd.DataFrame(data)
#     df = df.append(ds, ignore_index=True)
#     df.to_excel(excel_path, index=False, header=False)
#     f=open('computed.txt','a')
#     f.write(file_name+'\n')
#     f.close()
# file_list.close()